In [1]:
import polars
import hvplot.polars

import holoviews as hv

from polars import col

HEIGHT = 600
WIDTH = 1200

%opts magic unavailable (pyparsing cannot be imported)
%compositor magic unavailable (pyparsing cannot be imported)


In [7]:
amd = polars.read_csv("../tests/data/AMD.csv", try_parse_dates=True)
amd.head()

Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Time Zone
date,f64,f64,f64,f64,i64,f64,f64,str
1980-03-17,0.0,3.302083,3.125,3.145833,219600,0.0,0.0,"""EST"""
1980-03-18,0.0,3.125,2.9375,3.03125,727200,0.0,0.0,"""EST"""
1980-03-19,0.0,3.083333,3.020833,3.041667,295200,0.0,0.0,"""EST"""
1980-03-20,0.0,3.0625,3.010417,3.010417,159600,0.0,0.0,"""EST"""
1980-03-21,0.0,3.020833,2.90625,2.916667,130800,0.0,0.0,"""EST"""


In [8]:
import indicatory.means_medians as mms

In [23]:
amd_smas = mms.simple_moving_averages(amd, [10, 20, 60])
amd_macd = mms.moving_average_convergence_divergence(amd)
amd_mm = mms.moving_median(amd, window_size=5)

In [25]:
ohlc = amd.tail(50).hvplot.ohlc(x="Date", y=["Open", "High", "Low", "Close"], grid=True, height=HEIGHT, width=WIDTH)
smas_plot = amd_smas.tail(50).select(col(["Date", "SMA 10", "SMA 20", "SMA 60"])).hvplot.line(x="Date", y=["SMA 10", "SMA 20", "SMA 60"])

ohlc * smas_plot

:Overlay
   .Segments.I   :Segments   [Date,Low,Date,High]   (Open,High,Low,Close,lbound,ubound)
   .Rectangles.I :Rectangles   [lbound,Open,ubound,Close]   (Open,High,Low,Close,lbound,ubound)
   .NdOverlay.I  :NdOverlay   [Variable]
      :Curve   [Date]   (value)

In [22]:
mm_plot = amd_mm.tail(50).select(col(["Date", "MM 5"])).hvplot.step(x="Date", y=["MM 5"])

ohlc * mm_plot

:Overlay
   .Segments.I   :Segments   [Date,Low,Date,High]   (Open,High,Low,Close,lbound,ubound)
   .Rectangles.I :Rectangles   [lbound,Open,ubound,Close]   (Open,High,Low,Close,lbound,ubound)
   .Curve.I      :Curve   [Date]   (MM 5)

In [19]:
macd_plot = amd_macd.tail(50).select(col(["Date", "MACD 10/20", "SIG 5"])).hvplot.line(x="Date", y=["MACD 10/20", "SIG 5"], grid=True, height=int(HEIGHT / 4), width=WIDTH)

hv.Layout(ohlc + macd_plot).cols(1)

:Layout
   .Overlay.I   :Overlay
      .Segments.I   :Segments   [Date,Low,Date,High]   (Open,High,Low,Close,lbound,ubound)
      .Rectangles.I :Rectangles   [lbound,Open,ubound,Close]   (Open,High,Low,Close,lbound,ubound)
   .NdOverlay.I :NdOverlay   [Variable]
      :Curve   [Date]   (value)